In [4]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, types
from sqlalchemy.pool import NullPool
from hubspot.crm.contacts import SimplePublicObjectInput, ApiException
from hubspot.auth.oauth import ApiException
from validate_email import validate_email
import hubspot
import os
from dotenv import load_dotenv

load_dotenv()
DATABASE_CONNECTION_URI = os.environ["NEW_DB_URL"]
ACCESS_TOKEN = os.environ["ACCESS_TOKEN"]
client_id = os.environ["CLIENT_ID"]
client_secret = os.environ["CLIENT_SECRET"]

# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

class HubspotAPI:
    def __init__(self):
        # API KEY
        self.access_token = ACCESS_TOKEN
        self.client = hubspot.Client.create(access_token = self.access_token)
        self.max_results = 1000000

    def export_line_items(self, properties_dict):
        # Parameters
        results = []
        after_results = []
        previous_after = None
        after = 0

        while str(after).isnumeric():
            try:
                api_response = self.client.crm.deals.basic_api.get_page(
                    limit=100,
                    after=after,
                    properties=list(properties_dict.keys()),
                    associations=["line_items"],
                    archived=False,
                )
                api_response = api_response.to_dict()
                results.extend(api_response['results'])
                print("Hubspot Line Items Export has gathered " + str(len(results)) + " Line Items")
                try:
                    after = api_response['paging']['next']['after']
                except:
                    after = api_response['paging']
            except ApiException as e:
                print("Exception when calling basic_api->get_page: %s\n" % e)
        property_results = []
        for result in results:
            property_results.append(result["properties"])
        df = pd.DataFrame(property_results)
        # df.to_csv("raw_hubspot_companies.csv", encoding="latin-1", index=False, errors='ignore')
        #df.to_sql('hs_deals_dev_test', con=engine, if_exists='replace', index=False)
        assoc_results = []
        itterator = results.copy()
        for result in itterator:
            if result["associations"] is not None:
                lists_to_append = result["associations"]["line items"]["results"]
                to_append = []
                for i in lists_to_append:
                    dict_to_append = {}
                    dict_to_append["deal_id"] = result["id"]
                    dict_to_append["line_item_id"] = i["id"]
                    to_append.append(dict_to_append)
                assoc_results.append(to_append)
            else:
                assoc_results.append([{"deal_id": result["id"], "line_item_id": None}])
        deal_id = []
        line_item_id = []
        for index, item in enumerate(assoc_results):
            i = 0
            for a, b in item:
                deal_id.append(item[i][a])
                line_item_id.append(item[i][b])
                i += 1
        df = pd.DataFrame({"deal_id": deal_id, "line_item_id": line_item_id})
        df['deal_id'] = pd.to_numeric(df['deal_id'], errors='coerce').astype('Int64')
        df['line_item_id'] = pd.to_numeric(df['line_item_id'], errors='coerce').astype('Int64')
        # df.to_csv("hubspot_companies_to_deals.csv", encoding="latin-1", index=False, errors='ignore')
        df.to_sql('hubspot_deals_to_line_items', con=engine, if_exists='replace', index=False)
        

properties_dict = {

}

h = HubspotAPI()

h.export_line_items(properties_dict)

Hubspot Line Items Export has gathered 100 Line Items
Hubspot Line Items Export has gathered 200 Line Items
Hubspot Line Items Export has gathered 300 Line Items
Hubspot Line Items Export has gathered 400 Line Items
Hubspot Line Items Export has gathered 500 Line Items
Hubspot Line Items Export has gathered 600 Line Items
Hubspot Line Items Export has gathered 700 Line Items
Hubspot Line Items Export has gathered 800 Line Items
Hubspot Line Items Export has gathered 900 Line Items
Hubspot Line Items Export has gathered 1000 Line Items
Hubspot Line Items Export has gathered 1100 Line Items
Hubspot Line Items Export has gathered 1200 Line Items
Hubspot Line Items Export has gathered 1300 Line Items
Hubspot Line Items Export has gathered 1400 Line Items
Hubspot Line Items Export has gathered 1500 Line Items
Hubspot Line Items Export has gathered 1600 Line Items
Hubspot Line Items Export has gathered 1700 Line Items
Hubspot Line Items Export has gathered 1800 Line Items
Hubspot Line Items 